# Plot Types

This notebook demonstrates all the plot types available in xarray_plotly using real climate data from xarray's tutorial datasets.

In [ ]:
import xarray as xr

from xarray_plotly import config, xpx

config.notebook()  # Configure Plotly for notebook rendering

## Load Tutorial Datasets

We'll use two datasets:
- **air_temperature**: NCEP reanalysis with dimensions (time, lat, lon)
- **eraint_uvz**: ERA-Interim reanalysis with dimensions (month, level, latitude, longitude) - great for 4D data!

In [ ]:
# 3D dataset: time series of air temperature
ds_air = xr.tutorial.open_dataset("air_temperature")
air = ds_air["air"]
print(f"air_temperature: {dict(air.sizes)}")

# 4D dataset: atmospheric variables at different pressure levels
ds_era = xr.tutorial.open_dataset("eraint_uvz")
print(f"eraint_uvz variables: {list(ds_era.data_vars)}")
print(f"eraint_uvz dimensions: {dict(ds_era.sizes)}")

## Line Plot

Best for time series and continuous data:

In [ ]:
# Temperature over time at different latitudes
fig = xpx(air.sel(lat=[30, 45, 60], lon=260, method="nearest")).line(
    title="Air Temperature Time Series"
)
fig

### With markers

In [ ]:
# Subsample for visibility
fig = xpx(air.sel(lat=[30, 45, 60], lon=260, method="nearest").isel(time=slice(0, 50))).line(
    markers=True, title="Temperature with Markers"
)
fig

## Bar Chart

Best for comparing categorical data. Let's look at monthly mean temperature:

In [ ]:
# Monthly climatology at different locations
monthly = air.sel(lon=260, method="nearest").groupby("time.month").mean()
fig = xpx(monthly.sel(lat=[30, 45, 60], method="nearest")).bar(
    title="Monthly Mean Temperature by Latitude"
)
fig

### Grouped bars

In [ ]:
fig = xpx(monthly.sel(lat=[30, 45, 60], method="nearest")).bar(
    barmode="group", title="Grouped Monthly Temperature"
)
fig

## Area Chart

Best for showing composition over time:

In [ ]:
# Use wind components from ERA-Interim (take absolute values for stacking)
u_wind = abs(ds_era["u"].isel(month=0, level=0))
v_wind = abs(ds_era["v"].isel(month=0, level=0))

# Create a combined dataset for stacked area
wind_components = xr.concat(
    [u_wind.mean("longitude"), v_wind.mean("longitude")],
    dim=xr.Variable("component", ["U wind", "V wind"]),
)

fig = xpx(wind_components).area(title="Wind Components by Latitude")
fig

## Scatter Plot

Best for showing relationships between variables:

In [ ]:
# Temperature at different times
fig = xpx(air.sel(lat=[30, 45, 60], lon=260, method="nearest").isel(time=slice(0, 100))).scatter(
    title="Temperature Scatter"
)
fig

### Dimension vs Dimension (Geographic style)

You can plot one dimension against another, with values shown as color:

In [ ]:
# Geopotential at 500 hPa
geo = ds_era["z"].isel(month=0, level=1)  # level=1 is 500 hPa
fig = xpx(geo).scatter(
    x="longitude", y="latitude", color="value", title="Geopotential Height at 500 hPa"
)
fig

## Box Plot

Best for showing distributions:

In [ ]:
# Temperature distribution across different latitudes
# Sample time dimension to create distribution
fig = xpx(air.sel(lat=[25, 35, 45, 55, 65], lon=260, method="nearest")).box(
    title="Temperature Distribution by Latitude"
)
fig

## Heatmap (imshow)

Best for 2D grid data:

In [ ]:
# Air temperature at a single time step
fig = xpx(air.isel(time=0)).imshow(title="Air Temperature (2013-01-01)")
fig

### With different colorscale

In [ ]:
# U-wind component at 200 hPa (jet stream level)
u_200 = ds_era["u"].isel(month=0, level=0)  # level=0 is 200 hPa
fig = xpx(u_200).imshow(
    color_continuous_scale="RdBu_r",
    color_continuous_midpoint=0,
    title="U-Wind at 200 hPa (Jet Stream)",
)
fig

## Faceting

All plot types support faceting to create subplot grids. This is especially powerful with the 4D ERA-Interim data:

In [ ]:
# U-wind at different pressure levels and months
u_zonal = ds_era["u"].mean("longitude")  # Zonal mean

fig = xpx(u_zonal).line(
    x="latitude",
    color="level",
    facet_col="month",
    title="Zonal Mean U-Wind by Pressure Level",
)
fig

In [ ]:
# Heatmaps faceted by pressure level
fig = xpx(ds_era["u"].isel(month=0)).imshow(
    facet_col="level",
    color_continuous_scale="RdBu_r",
    color_continuous_midpoint=0,
    title="U-Wind at Different Pressure Levels",
)
fig

## Animation

Create animated plots by assigning a dimension to `animation_frame`:

In [ ]:
# Animate through pressure levels
fig = xpx(ds_era["z"].isel(month=0)).imshow(
    animation_frame="level",
    title="Geopotential Height by Pressure Level",
    color_continuous_scale="Viridis",
)
fig

In [ ]:
# Monthly temperature climatology animation
monthly_map = air.groupby("time.month").mean()

fig = xpx(monthly_map).imshow(
    animation_frame="month",
    title="Monthly Mean Air Temperature",
    color_continuous_scale="thermal",
)
fig